## The MemProtMD API
The MemProtMD API connects the MemProtMD database to applications such
as this site. The API is free to use for any application, provided data
obtained through it is properly attributed.

### General notes
We recommend using [Postman](https://www.getpostman.com/) to check that
the API methods shown here behave as expected. Postman can also output
code snippets in a number of languages for pipeline applications. Examples
in this guide use Python's Requests library.

### The MemProtMD Data Model

MemProtMD deals with a few different data types. It's worth understanding what these are before diving in:

#### Simulation

Each **simulation** is a simulation set up and run through the MemProtMD pipeline. The simulation has a Coarse-Grained Self-Assembley, Coarse-Grained Dynamics and Atomistic stage. Each simulation is linked to one **reference**.

#### Reference

Each **reference** is a reference to an accession in an external database, such as the Protein DataBank. It will have a unique identifier, which is the database abbreviation (e.g. PDB), the pipe (|) symbol and the accession code from that database. Record 1a0s from the Protein DataBank is therefore `PDB|1a0s`. Records typically also have a title, list of simulation IDs, `parent` reference (where the reference falls into a tree structure), `member_of` list (if it is a member of a broader database reference) and a `collection_members` list (the inverse of the `member_of` list).

## The MemProtMD JSON/MongoDB API

The MemProtMD JSON/MongoDB API aims to securely expose as much functionality as possible from the underlying MongoDB collections. Data is sent to the server in the body of an HTTP POST request as JSON and returned via HTTP response as JSON. The status of the response will be `200` for sucessful requests or `400` for unsuccessful requests. The reason for failure should be detailed in the JSON response.

There are three collections which can be queried using the JSON API - `simulations`, `refs` and `authors`.

First off, we'll import the Python requests library and set the root URI for the MemProtMD site.

In [ ]:
import requests

In [ ]:
MEMPROTMD_ROOT_URI = "http://memprotmd.bioch.ox.ac.uk/"

#### Bulk Operations

Often you might want to do grab a complete list of references or simulations from MemProtMD. These come with only the most commonly used fields so you may need to make additional requests (see below) to fill in details for other fields.

**All simulations** POST to http://memprotmd.bioch.ox.ac.uk/api/simulations/all with no parameters.

**All references** POST to http://memprotmd.bioch.ox.ac.uk/api/references/all/ with no parameters. You can optionally restrict this list to specific databases by adding the database name to the end of the URI, e.g. http://memprotmd.bioch.ox.ac.uk/api/references/all/Pfam for the Pfam database.

These Python examples will show only the first item from the returned list:

In [ ]:
def get_all_memprotmd_simulations():
    return requests.post(MEMPROTMD_ROOT_URI + "api/simulations/all").json()

get_all_memprotmd_simulations()[0]

In [ ]:
def get_all_memprotmd_references(database=""):
    return requests.post(MEMPROTMD_ROOT_URI + "api/references/all/" + database).json()

get_all_memprotmd_references("GO")[0]

#### Full data for a reference or simulation

In this case the full database record is returned.

**Full data for a simulation** POST to http://memprotmd.bioch.ox.ac.uk/api/simulations/[sim_id]

**Full data for a reference** POST to http://memprotmd.bioch.ox.ac.uk/api/references/[database_id]/[sim_id]

The Python examples below show how you'd use this in practice:

In [ ]:
def get_memprotmd_simulation(sim_id):
    return requests.post(
        MEMPROTMD_ROOT_URI 
        + "api/simulations/{sim_id}".format(
            sim_id = sim_id
        )).json()
get_memprotmd_simulation("1a0s_default_dppc")

In [ ]:
def get_memprotmd_reference(db, accession):
    return requests.post(
        MEMPROTMD_ROOT_URI 
        + "api/references/{db}/{accession}".format(
            db=db, accession=accession
        )).json()
get_memprotmd_reference("PDB", "4wit")

You can also pass in a list of references to get more detailed (although not full) data:

In [ ]:
def get_memprotmd_reference_list(list_of_references):
    return requests.post(
        
        MEMPROTMD_ROOT_URI 
        + "api/references/from_list",
        
        json={"ids" : list_of_references}
        
    ).json()
get_memprotmd_reference_list(["PDB|1a0s", "PDB|1bcc"])

#### Searching MemProtMD

**Simple Search** POST to http://memprotmd.bioch.ox.ac.uk/api/search/simple with a JSON object in the POST body. The JSON object should contain a `searchTerm` (the string to search for) and may contain `numResults` - the number of results to send, `inDatabases` - a list of databases to look in and `sizeBias`, which controls how much the number of members a reference has should increase its relevance. Higher values will bias the search towards references with more members.

In [ ]:
def memprotmd_text_search(
            search_term, 
            in_databases=["PDB", "mpm", "TCDB", "GO", "Pfam"], 
            size_bias=0.1,
            n_results=10):
    
    return requests.post(
        
        MEMPROTMD_ROOT_URI 
        + "api/search/simple",
        
        json={
            "searchTerm" : search_term,
            "inDatabases" : in_databases,
            "numResults" : n_results,
            "sizeBias" : size_bias
        }
        
    ).json()

memprotmd_text_search("gpcr", n_results=2, size_bias=10)

**Advanced Search** This is where things get fun (for MongoDB nerds like Tom). Advanced Search allows you to directly query the MongoDB dataset with a few restrictions:

- We only permit a few dollar operators to be used. If you'd like to use one we're currently blocking, get in touch and we'll consider adding it. At present we allow `$or`, `$and`, `$ne`, `$eq`, `$lt`, `$gt`, `$not`, `$nin`, `$in` and `$elemMatch`.

- You'll only be able to access the **refs**, **simulations** and **authors** collections. Sadly we've had to make the **shocking_secrets**, **banking_passwords** and **unpublished_structures** collections unavailable for the moment.

- **We're watching you.** Be kind to our server or you'll be blocked for a few minutes.

There are four parts to an advanced search. These should be familiar to anyone who's used MongoDB before:

- Name of the collection to query
- Query object
- Projection object
- Options object (e.g. sort, limit and skip)

As an example, let's find all the records with a chain containing seven transmembrane alpha helices, return only the id, chainID, number of alpha helical elements and number of lipid molecules, then sort by the number of lipid molecules (ascending) and show the first one.

In [ ]:
def memprotmd_advanced_search(collection_name, query, projection, options):
    
    return requests.post(
        
        MEMPROTMD_ROOT_URI 
        + "api/search/advanced",
        
        json={
            "collectionName" : collection_name,
            "query" : query,
            "projection" : projection,
            "options" : options
        }
        
    ).json()

memprotmd_advanced_search(
    
    # Use the simulation collection
    "simulations",
    
    # Look in the chains array of each simulation and see if
    # any element in the array has the field tm_alpha equal to 7
    {
        "chains" : {
            "$elemMatch" : {
                "tm_alpha" : 7
            }
        }
    },
    
    # Projection - choose the fields to return. ID is returned
    # by default.
    {
        "chains.tm_alpha" : 1, 
        "chains.chainID" : 1, 
        "sim_validate.n_lipid_molecules" : 1
    },
    
    # Options - sort and then limit
    {
        "sort" : {
            "sim_validate.n_lipid_molecules" : 1
        }, 
        "limit" : 1
    }
)

Alternatively let's have an open query and projection which includes everything. I'm kind to servers and don't like being blocked so I only return the first one (you can definitely get away with occasionally pulling the entire dataset, but please limit your queries where possible!).

In [ ]:
memprotmd_advanced_search(
    "refs",
    {},
    {},
    {"limit" : 1})

In [ ]:
memprotmd_advanced_search(
    "simulations",
    {},
    {},
    {"limit" : 1})

In [ ]:
memprotmd_advanced_search(
    "authors",
    {},
    {},
    {"limit" : 1})

If it didn't work our server will be as helpful as possible:

In [ ]:
memprotmd_advanced_search(
    "authors",
    {"$doSomethingShady" : "secrets"},
    {},
    {"limit" : 1})

### Data API

The Data API behaves like a normal filesystem (although we don't provide anything that acts like an index). Make an HTTP GET request and we'll send you the files. These examples use `matplotlib` and `pandas`.

In [ ]:
% matplotlib inline
import pandas

Let's have some fun and download the postprocessed (annotated) contacts file for 1a0s_default_dppc, plot a histogram of temperature (&beta;) factors and a line graph of contacts along the amino acid sequence.

In [ ]:
def get_resid_contacts(sim_id):
    return pandas.read_csv(
        MEMPROTMD_ROOT_URI 
        + "data/memprotmd/simulations/"
        + sim_id
        + "/files/contacts/by_resid_postprocess.csv"
    )
contacts = get_resid_contacts("1a0s_default_dppc")

contacts.hist("tempFactor")
contacts.plot(
    y=["group=Tail", "group=Head"], 
    x=["resSeq"], 
    color=["orange", "green"],
    figsize=[15,4])

You can also load structures from MemProtMD using mdtraj (or similar product):

In [ ]:
import mdtraj as md
import nglview as nv

In [ ]:
def mdtraj_load_memprotmd(sim_id, structure_name):
    traj = md.load_pdb( MEMPROTMD_ROOT_URI 
                + "data/memprotmd/simulations/"
                + sim_id
                + "/files/structures/"
                + structure_name
                + ".pdb")
    return traj

In [ ]:
traj = mdtraj_load_memprotmd("1afo_default_dppc", "at")

In [ ]:
protein_view = nv.show_mdtraj(traj.atom_slice(traj.topology.select("resid < 40")))
protein_view.add_ball_and_stick(opacity=0.5)
protein_view

In [ ]:
cg_traj = mdtraj_load_memprotmd("1afo_default_dppc", "cg")

In [ ]:
cg_view = nv.show_mdtraj(cg_traj)
cg_view.add_hyperball(radius=4, shrink=0.44, scale=0.6)
cg_view

You should now have most of the materials you need for some bioinformatics fun!